# Moca.jl tutorial

A hands-on guide to the Moca.jl Monte Carlo simulation engine.

## Overview

Moca.jl provides:

- Thread-safe random number generation
- Stochastic process implementations (Wiener, GBM)
- Efficient path generation (streaming and stored modes)
- Online statistics with Welford's algorithm


In [1]:
# Load the module
include("./src/Moca.jl")
using Main.Moca

## Random number generation

Thread-safe RNG with independent stream splitting for parallel execution.


In [2]:
# Create a seeded RNG
rng = StandardRNG(42)

# Sample from distributions
x = rand_normal(rng; mean=0.0, std_dev=2.0)
y = rand_uniform(rng; left=-1.0, right=1.0)

println("Normal sample: $x")
println("Uniform sample: $y")

Normal sample: 2.420564044098074
Uniform sample: -0.8710294978033466


In [3]:
# Split into independent streams
rng_streams = split_rng(rng, 3)
println("Created $(length(rng_streams)) independent RNG streams")

for (i, stream) in enumerate(rng_streams)
    val = rand_normal(stream)
    println("Stream $i: $val")
end

Created 3 independent RNG streams
Stream 1: -0.3039639640846837
Stream 2: 0.2584242109761766
Stream 3: -2.4164144467210846


## Stochastic processes

### Wiener process

Brownian motion with drift: `dX = drift*dt + volatility*dW`


In [4]:
# Define and step
wiener = WienerProcess(drift=0.1, volatility=0.2)
rng = StandardRNG(123)

x0 = 0.0
x1 = Moca.step(wiener, x0, 0.1, rng)
println("Wiener process: $x0 -> $x1")

Wiener process: 0.0 -> 0.03046145851249486


### Geometric Brownian motion

`dS = drift*S*dt + volatility*S*dW`


In [5]:
# Define and step
gbm = GeometricBrownianMotion(drift=0.05, volatility=0.3)
rng = StandardRNG(456)

s0 = 100.0
s1 = Moca.step(gbm, s0, 0.1, rng)
println("GBM: $s0 -> $s1")

GBM: 100.0 -> 98.88742512449822


## Path generation

Two modes:

- **Stored** (`store_path=true`): Returns full path array
- **Streaming** (`store_path=false`): Returns only terminal value (memory efficient)


In [6]:
# Stored mode
config_stored = PathConfig(100, 0.01, true)
rng = StandardRNG(789)
path = generate_path(wiener, 0.0, config_stored, rng)

println("Generated path with $(length(path)) points")
println("Initial: $(path[1]), Terminal: $(path[end])")

Generated path with 101 points
Initial: 0.0, Terminal: -0.0790234636576309


In [7]:
# Streaming mode
config_streaming = PathConfig(100, 0.01, false)
rng = StandardRNG(202)
terminal = generate_path(wiener, 0.0, config_streaming, rng)

println("Terminal value: $terminal")

Terminal value: 0.3518924818681834


## Online statistics

Compute statistics incrementally using Welford's algorithm.


In [8]:
# Simple example
stats = OnlineStatistics()
for val in [1.0, 2.0, 3.0, 4.0, 5.0]
    update!(stats, val)
end

result = Moca.finalize(stats)
println("Mean: $(result.mean)")
println("Std dev: $(result.std_dev)")
println("95% confidence interval: $(result.confidence_interval)")

Mean: 3.0
Std dev: 1.5811388300841898
95% confidence interval: (1.6140961756503196, 4.385903824349681)


### Recovering distribution parameters


In [9]:
# Sample from N(10, 2^2) and verify recovery
stats_mc = OnlineStatistics()
rng = StandardRNG(303)

for i in 1:10_000
    sample = rand_normal(rng; mean=10.0, std_dev=2.0)
    update!(stats_mc, sample)
end

result = Moca.finalize(stats_mc)
println("True mean: 10.0, Estimated: $(result.mean)")
println("True std: 2.0, Estimated: $(result.std_dev)")

True mean: 10.0, Estimated: 10.001229325877969
True std: 2.0, Estimated: 1.9938835555097933


### Aggregating from multiple workers


In [10]:
# Simulate 3 workers
worker_stats = OnlineStatistics[]

for (i, n_samples) in enumerate([100, 150, 200])
    stats = OnlineStatistics()
    rng = StandardRNG(400 + i)
    for _ in 1:n_samples
        update!(stats, rand_normal(rng; mean=5.0, std_dev=1.0))
    end
    push!(worker_stats, stats)
end

# Combine using Chan's algorithm
combined = aggregate(worker_stats)
result = Moca.finalize(combined)

println("Total samples: $(result.n_samples)")
println("Mean: $(result.mean) (expected ~5.0)")
println("Std: $(result.std_dev) (expected ~1.0)")

Total samples: 450
Mean: 5.104965453305758 (expected ~5.0)
Std: 0.9719626521778952 (expected ~1.0)


## Monte Carlo simulation

The `simulate()` function orchestrates the full workflow.


### Estimating terminal values


In [11]:
# Estimate E[X_T] for Wiener process
# Theoretical: E[X_T] = initial + drift*T = 0 + 0.1*1.0 = 0.1
wiener = WienerProcess(drift=0.1, volatility=0.2)
config = PathConfig(100, 0.01, false)  # T = 1.0

result = simulate(
    wiener,
    x -> x,  # Identity functional
    10_000;
    initial_state=0.0,
    config=config,
    rng_seed=42
)

println("Theoretical: 0.1")
println("Estimated: $(result.mean)")
println("95% confidence interval: $(result.confidence_interval)")

Theoretical: 0.1
Estimated: 0.1013682300489188
95% confidence interval: (0.09741478296830754, 0.10532167712953007)


### Path-dependent functionals


In [12]:
# Estimate E[max(S)] for GBM
gbm = GeometricBrownianMotion(drift=0.05, volatility=0.3)
config = PathConfig(100, 0.01, true)  # Need full path

result = simulate(
    gbm,
    path -> maximum(path),
    10_000;
    initial_state=100.0,
    config=config,
    rng_seed=123
)

println("Estimated E[max(S)]: $(result.mean)")
println("Std error: $(result.std_err)")
println("95% confidence interval: $(result.confidence_interval)")

Estimated E[max(S)]: 127.6700481905793
Std error: 0.26066947025399734
95% confidence interval: (127.15914541701233, 128.18095096414626)


### Custom functionals


In [13]:
# Call option: E[max(S_T - K, 0)]
gbm = GeometricBrownianMotion(drift=0.05, volatility=0.3)
config = PathConfig(252, 1 / 252, false)  # 1 year, daily steps
strike = 100.0

result = simulate(
    gbm,
    s -> max(s - strike, 0.0),  # Payoff function
    50_000;
    initial_state=100.0,
    config=config,
    rng_seed=456
)

println("Call option value: $(result.mean)")
println("95% confidence interval: $(result.confidence_interval)")

Call option value: 15.07549767882585
95% confidence interval: (14.868037382798224, 15.282957974853474)


## Parallel execution

Multi-threaded execution using `parallel=:threads` for faster simulations.


In [14]:
# Check available threads
println("Available threads: $(Threads.nthreads())")

Available threads: 8


### Scalability across problem sizes

Threading overhead is amortized over larger problems, leading to better speedups.


In [15]:
# Test with different problem sizes
using Printf

for n_paths in [10_000, 50_000, 100_000, 500_000]
    println("\n=== Testing with $n_paths paths ===")

    process = WienerProcess(drift=0.1, volatility=0.2)
    config = PathConfig(100, 0.01, false)

    # Serial
    t_serial = @elapsed simulate(
        process, x -> x, n_paths;
        initial_state=1.0, config=config, rng_seed=42, parallel=:serial
    )

    # Threaded
    t_threads = @elapsed simulate(
        process, x -> x, n_paths;
        initial_state=1.0, config=config, rng_seed=42, parallel=:threads
    )

    speedup = t_serial / t_threads
    @printf("Serial: %.3f s, Threads: %.3f s, Speedup: %.2fx\n", t_serial, t_threads, speedup)
end


=== Testing with 10000 paths ===
Serial: 0.469 s, Threads: 0.162 s, Speedup: 2.90x

=== Testing with 50000 paths ===
Serial: 1.806 s, Threads: 0.051 s, Speedup: 35.62x

=== Testing with 100000 paths ===
Serial: 2.938 s, Threads: 0.177 s, Speedup: 16.59x

=== Testing with 500000 paths ===
Serial: 20.253 s, Threads: 1.533 s, Speedup: 13.21x


### Performance with path-dependent functionals


In [16]:
# More expensive computation: max over stored paths
gbm = GeometricBrownianMotion(drift=0.05, volatility=0.3)
config_stored = PathConfig(252, 1 / 252, true)
n_paths = 50_000

println("Serial execution (with path storage):")
@time result_serial = simulate(
    gbm,
    path -> maximum(path),
    n_paths;
    initial_state=100.0,
    config=config_stored,
    rng_seed=789,
    parallel=:serial
)

println("\nThreaded execution (with path storage):")
@time result_threads = simulate(
    gbm,
    path -> maximum(path),
    n_paths;
    initial_state=100.0,
    config=config_stored,
    rng_seed=789,
    parallel=:threads
)

println("\n--- Results ---")
println("Serial:  $(result_serial.mean) ± $(result_serial.std_err)")
println("Threads: $(result_threads.mean) ± $(result_threads.std_err)")

Serial execution (with path storage):
  7.043088 seconds (14.27 M allocations: 1.544 GiB, 70.42% gc time, 1.99% compilation time)

Threaded execution (with path storage):
  0.344200 seconds (13.10 M allocations: 590.854 MiB, 11.17% gc time, 140.30% compilation time)

--- Results ---
Serial:  128.38256660533156 ± 0.1157413892396172
Threads: 128.31818207663176 ± 0.11601398174615529


## Summary

1. Thread-safe RNG with stream splitting
2. Wiener and GBM process implementations
3. Streaming and stored path generation
4. Online statistics with Welford's algorithm
5. Statistics aggregation with Chan's algorithm
6. Full Monte Carlo simulation with custom functionals
7. Multi-threaded execution for performance scaling
